### Climatelearn testing 

In [ ]:
# This jupyter notebook uses climatelearn tools 
# I use the same data from WeatherBench and compare the results with my implementation of ViT

In [ ]:
import sys
print(sys.version)

In [ ]:
import torch as T
import torch.nn as nn
#from torchtext import data, datasets
#from torchtext.vocab import Vocab
import torch.optim as optim
import time
import copy
import torch
import torch.nn.functional as F
from torchsummary import summary
import math

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import math
import datetime
from collections import OrderedDict

# Custom imports
from utils.datagenerator import *
from utils.util_data import * 
from utils.metrics import *
from utils.helpers import *
from utils.drop import *

In [ ]:
print("Cuda Avaliable :", torch.cuda.is_available())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from climate_learn.models import load_model

In [ ]:
# open data from WeatherBench
DATADIR = '/storage/homefs/no21h426/WeatherBench-master/data/WeatherBench/5.625deg/'
# Load the entire dataset
z500 = xr.open_mfdataset(f'{DATADIR}geopotential_500/*.nc', combine='by_coords').z
t850 = xr.open_mfdataset(f'{DATADIR}temperature_850/*.nc', combine='by_coords').t.drop('level')
ds = xr.merge([z500, t850])
# only load a subset of the training data
ds_train = ds.sel(time=slice('2015', '2016'))  
ds_test = ds.sel(time=slice('2017', '2018'))

In [ ]:
# then we need a dictionary for all the variables and levels we want to extract from the dataset
dic = OrderedDict({'z': None, 't': None})
lead_time =1
bs = 32
# Create a training and validation data generator. Use the train mean and std for validation as well.
dg_train = DataGenerator(
    ds_train.sel(time=slice('2015', '2015')), dic, lead_time, batch_size=bs, load=True)
dg_valid = DataGenerator(
    ds_train.sel(time=slice('2016', '2016')), dic, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, shuffle=False)

In [ ]:
## Download from climatelearn
from climate_learn.data import download

In [ ]:
model_kwargs = {
    "in_channels": 2,
    "out_channels": 2,
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
}


In [ ]:
model_module = load_model(
    name="vit",
    task="forecasting",
    model_kwargs=None,
    optim_kwargs=optim_kwargs
)

